In [1]:
import requests
from bs4 import BeautifulSoup
import re

# url = 'https://hinative.com/questions/23852396'
url = 'https://hinative.com/questions/23852149'

# Define the phrase to search for in the questions
search_phrase = 'Does this sound natural?'

# Send a GET request to the URL and parse the HTML content using BeautifulSoup
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Find the title tag and check if it contains the search phrase
title_tag = soup.find('title')

if search_phrase in title_tag.string:
    # Extract the question title from the title tag
    question_title = title_tag.string.replace(search_phrase, '').strip()

    # Extract the responses that include the search phrase
    responses = soup.find_all('div', class_='text_correction_answer')

    # Loop through the responses and extract the original and corrected phrases
    for response in responses:

        original_element = response.find_all('p')

        for i in range(1, len(original_element), 2):
            element = original_element[i]
            # Extract text content of corrected phrase from HTML
            original_phrase = ''
            for child in element.children:
                if child.name == 'del':
                    original_phrase += child.text
                elif child.name == 'ins':
                    continue
                else:
                    original_phrase += child.string
            print(original_phrase)

        corrected_element = response.find_all('p')
        for i in range(1, len(corrected_element), 2):
            element = corrected_element[i]
            # Extract text content of corrected phrase from HTML
            corrected_phrase = ''
            for child in element.children:
                if child.name == 'ins':
                    corrected_phrase += child.text
                elif child.name == 'del':
                    continue
                else:
                    corrected_phrase += child.string
            print(corrected_phrase)


1 Hi, Recently I started going to yoga classes.
It was hot yoga the other day,
I went to
today!
3 Where is the Ocean location in the video you sent me earlier?
I would like to go there next time I have a chance.
1 Hi, I recenly started going to yoga classes.
It was hot yoga the other day.
I went to
today!
3 Where is the Ocean located in the video you sent me earlier?
I would like to go there next time I have a chance.


In [3]:
import requests
from bs4 import BeautifulSoup
import re
import csv

url = 'https://hinative.com/questions/23852396'

# Define the phrase to search for in the questions
search_phrase = 'Does this sound natural?'

# Send a GET request to the URL and parse the HTML content using BeautifulSoup
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Find the title tag and check if it contains the search phrase
title_tag = soup.find('title')

if search_phrase in title_tag.string:
    # Extract the question title from the title tag
    question_title = title_tag.string.replace(search_phrase, '').strip()

    # Extract the responses that include the search phrase
    responses = soup.find_all('div', class_='text_correction_answer')

    # Open a CSV file for writing
    with open('learner_corpus.csv', 'w', newline='') as file:
        writer = csv.writer(file)

        # Write the header row to the CSV file
        writer.writerow(['Original', 'Corrected'])

        # Loop through the responses and extract the original and corrected phrases
        for response in responses:
            original_element = response.find_all('p')

            for i in range(1, len(original_element), 2):
                element = original_element[i]
                # Extract text content of corrected phrase from HTML
                original_phrase = ''
                for child in element.children:
                    if child.name == 'del':
                        original_phrase += child.text
                    elif child.name == 'ins':
                        continue
                    else:
                        original_phrase += child.string

                corrected_element = original_element[i+1]
                # Extract text content of corrected phrase from HTML
                corrected_phrase = ''
                for child in corrected_element.children:
                    if child.name == 'ins':
                        corrected_phrase += child.text
                    elif child.name == 'del':
                        continue
                    else:
                        corrected_phrase += child.string

                # Write the original and corrected phrases to the CSV file
                writer.writerow([original_phrase.strip(), corrected_phrase.strip()])
